In [ ]:
import sys, struct, logging
import io, time, imageio, os
from datetime import timedelta
from PIL import Image, ImageEnhance
import numpy as np

In [ ]:
class SeqFileFormat:
    HEADER={
        'numOfFrames':  {'offset':572, 'L':4, 'format':'=L'},
        'headerSize':   {'offset': 32, 'L':4, 'format':'=l'},
        'fixedSize':    {'offset':600, 'L':4, 'format':'=L'},
        'timeOffset':   {'offset':612, 'L':4, 'format':'=l'},
        'compression':  {'offset':620, 'L':4, 'format':'=l'},
    }
    
    TIMESTAMP={
        'offset':0,
        'L':8,
        'format':'=lHH'
    }
    
    FRAMERATE={
        'slow':20,
        'fast':200
    }
    
    def __init_subclass__(cls,*args):
        assert hasattr(cls, '__enter__'), f"{cls.__name__} is not a context manager!"
        
    def _read_frame_size(self):
        fameSizeByte=self.file.read(4)
        
        return int.from_bytes(fameSizeByte,sys.byteorder)-4
    
    def _read_timestamp(self):
        timestampByte=self.file.read(self.TIMESTAMP['L'])
        timestamp=struct.unpack(self.TIMESTAMP['format'],timestampByte)
        
        time=timedelta(seconds=timestamp[0], milliseconds=timestamp[1], microseconds=timestamp[2])
        
        return time

In [ ]:
class SeqFile(SeqFileFormat):
    """
    This class implements basic methods to extract single frames from a 
    norpix *.SEQ movie file WITH jpeg compression.
    Interface:
    with SeqFile(path_to_file) as f:
        f.goto_time (time_to_go_to_as_a_deltatime_object) OR f.goto_frame(frame_number)
        frameImage, timeStamp = f.get_frame()
    """

    def __init__(self, filePath):
        self.filePath=filePath
        
    def __len__(self):
        return self.num_of_frames()
        

    @classmethod
    def open(cls, *arg, **kw):
        obj=cls(*arg, **kw)
        return obj.__enter__()
        
    def _peek(self, size, offset, from_what=0):
        """
        read a few bytes from an offset without moving the file pointer
        """
        f_init=self.file.tell()
        try:
            self.file.seek(offset, from_what)
            data=self.file.read(size)
        finally:
            self.file.seek(f_init)
            
        return data
        
    def __enter__(self):
        try:
            self.file = open(self.filePath,'rb')
            self.frame=np.inf
            self.time = timedelta.max
        except Exception as e:
            logging.error(f"file: {self.filePath} failed to open!\n{repr(e)}")
            return None
        
        return self

    def __exit__(self, *args):
        try:
            self.file.close()
        except AttributeError:
            #when there is an error in opening the file, so there is no close() method
            pass
    
    def num_of_frames(self):
        data= self._peek(self.HEADER['numOfFrames']['L'],self.HEADER['numOfFrames']['offset'])
        return struct.unpack(self.HEADER['numOfFrames']['format'],data)[0]
    
    def time_offset(self):
        """
        this method returns the timestamp of the first frame
        """
        headerByte=self._peek(self.HEADER['headerSize']['L'],self.HEADER['headerSize']['offset'])
        headerSize=struct.unpack(self.HEADER['headerSize']['format'],headerByte)[0]

        frame1sizeByte=self._peek(4,headerSize)
        frame1size=int.from_bytes(frame1sizeByte,sys.byteorder)-4

        timeByte=self._peek(self.TIMESTAMP['L'],headerSize+4+frame1size)
        timestamp=struct.unpack(self.TIMESTAMP['format'],timeByte)
        
        time=timedelta(seconds=timestamp[0], milliseconds=timestamp[1], microseconds=timestamp[2])
        return time

    def get_frame(self):
        """
        this method yields one frame image.
        the pointer must be at the begining of a frame (frame size byte)
        reaching the end will raise an error
        """
        frameSize=self._read_frame_size()
        imageByte=self.file.read(frameSize)
        
        image = Image.open(io.BytesIO(imageByte))
        timestamp=self._read_timestamp()-self.time_offset()
        
        self.frame+=1
        self.time=timestamp
        return image, timestamp

    def goto_frame(self,N=1):
        """
        This method browses the pointer to the begining of the N_th frame
        """
        headerByte=self._peek(self.HEADER['headerSize']['L'],self.HEADER['headerSize']['offset'])
        headerSize=struct.unpack(self.HEADER['headerSize']['format'],headerByte)[0]
        #pre-allocation to incease the speed
        timestampL=self.TIMESTAMP['L']
        byteorder= sys.byteorder
        file=self.file
        
        if N<self.frame:
            n=1
            file.seek(headerSize)
        elif N==self.frame:
            return
        else:
            n=self.frame
        while n<N:
            fameSizeByte=file.read(4)
            frameSize= int.from_bytes(fameSizeByte,byteorder)-4
            file.seek(frameSize+timestampL,1)
            n+=1
        logging.info(f'current frame:{n}')
        self.frame=N
        
        timeByte=self._peek(self.TIMESTAMP['L'],-self.TIMESTAMP['L'],1)
        timestamp=struct.unpack(self.TIMESTAMP['format'],timeByte)        
        time=timedelta(seconds=timestamp[0], milliseconds=timestamp[1], microseconds=timestamp[2])-self.time_offset()
        self.time =time
            
    def goto_time(self,T=0):
        """
        this method browses the file to the begining of a frame, closest to the time T
        """
        if T==0:
            T=self.time_offset()
        
        assert isinstance(T,timedelta), 'Bad time format'
        
        if T.days >= 1: #in case T is not relative to first frame
            T=T-self.time_offset()
            
        headerByte=self._peek(self.HEADER['headerSize']['L'],self.HEADER['headerSize']['offset'])
        headerSize=struct.unpack(self.HEADER['headerSize']['format'],headerByte)[0]
        #pre-allocation to incease the speed
        timestampL     =self.TIMESTAMP['L']
        timestampFormat=self.TIMESTAMP['format']
        byteorder= sys.byteorder
        file=self.file
        refTime=self.time_offset()
        
        if T< self.time:
            dt=timedelta(days=2)
            frame=1
            file.seek(headerSize)
        elif T==self.frame:
            return
        else:
            frame=self.frame
            dt=T-self.time
        
        zero=timedelta()
        while dt>=zero:
            fameSizeByte=file.read(4)
            frameSize= int.from_bytes(fameSizeByte,byteorder)-4
            file.seek(frameSize,1)
            timeByte=file.read(timestampL)
            timestamp=struct.unpack(timestampFormat,timeByte)
            time=timedelta(seconds=timestamp[0], milliseconds=timestamp[1], microseconds=timestamp[2])-refTime
            dt=T-time
            frame+=1
        
        file.seek(-(frameSize+timestampL+4),1)
        logging.info(f'current time:{time}')
        self.time=time
        self.frame=frame
    

This class processes the video

In [ ]:
class SeqProcess:
    """
    This class will implement methods to extract multiple frames and save them as viedos etc.
    """
    def __init__(self,seq):
        assert isinstance(seq,SeqFile), "Input must be a SeqSile object"
        self.file=seq
        try:
            evt=self.read_file(seq.filePath.rstrip('.seq')+'.evt.cam')
            self.camEvent=tuple(evt)
        except:
            self.camEvent=None
    
    def video_epoch(self, t1, t2, fps):
        """
        extracts the video betweeen t1 and t2, and returns the images
        corresponding to fps, fps must be smaller than the video's
        t1,t2: with reference to the Ephy recording (based on .evt.cam file)
        """
        assert isinstance(t1,timedelta), 'Bad time format'
        assert t2>t1, "t2 must be bigger than t1"
        assert fps < self.file.FRAMERATE['fast'], 'FPS too high'
        
        self._time_delay()
        T1=t1-self._delay
        T2=t2-self._delay
        interval=timedelta(seconds=1/fps)
        fix=self.adj_contrast_brightness
        
        self.file.goto_time(T1)
        im,_=self.file.get_frame()
        im_list=[fix(im)]
        t=T1
        while t<T2:
            self.file.goto_time(t+interval)
            im,frame_time=self.file.get_frame()
            im_list.append(fix(im))
            t=frame_time
        
        return tuple(im_list)
        
    def _time_delay(self):
        try:
            return self._delay
        except AttributeError:
            #first trial = first frame of the camera
            self._delay=timedelta(milliseconds=self.camEvent[0])
            return self._delay
        
    @staticmethod
    def read_file(path):
        """reads a one column text file"""
        if not os.path.exists(path):
            logging.error(f"No file {path}")
            raise IOError
        
        a=[]
        with open(path,'rt') as f:
            for line in f:
                a.append(float(line.rstrip('\n').split()[0]))
        return a

    @classmethod
    def save_gif(cls, ims, filepath, duration=1):
        """
        ims: a list of all Image objects
        duration: a list of the duration each image will be displayed
        """

        assert filepath.endswith('.gif') or filepath.endswith('.GIF'), "Bad file name"
        assert hasattr(ims,'__iter__'), 'images not in an iterator'
    #     assert len(duration)==len(ims) or len(duration)==1

        with imageio.get_writer(filepath, mode='I', format='.gif',
                                subrectangles= True, duration=duration) as writer:
            for im in ims:
                writer.append_data(np.asarray(im))
    
    @staticmethod
    def adj_contrast_brightness(im,cont_fac=1.5,bright_fac=40):
        """Basic Brightness and Contrast adjustment of the images"""
        
#         if not isinstance(im,Image.Image):
#             raise TypeError(f'input must be a PIL.Image object, instead of {im}')

        a=ImageEnhance.Brightness(im)
        im2=a.enhance(bright_fac)
        a=ImageEnhance.Contrast(im2)
        im3=a.enhance(cont_fac)

        return im3

In [102]:
if "__file__" not in dir():
    filePath="/data/Rat106/Experiments/Rat106_2017_03_31_10_56/Rat106_2017_03_31_10_56.seq"
    filePath="/data/Rat174/Experiments/Rat174_2018_05_02_14_29/Rat174_2018_05_02_14_29.seq"
    filePath="/data/Rat172/Experiments/Rat172_2018_02_26_18_15/Rat172_2018_02_26_18_15.seq"
    fix=SeqProcess.adj_contrast_brightness
#     t0=time.perf_counter()
    ims=[]
    with SeqFile(filePath) as f:
        process=SeqProcess(f)
        print(len(f))
        ims=process.video_epoch(timedelta(milliseconds=67509.25),timedelta(milliseconds=72509.25),25)
#         f.goto_time(timedelta(minutes=2.3))
#         print(f"frame {f.frame}")
#         print(f"time {f.time}")
#         f.time=5
#         print(f"time {f.time}")
#         for i in range(1000):
#             t=f.time_offset()
#             T=t+timedelta(minutes=30,seconds=i/10)
#             f.goto_time(T)
#             im,_=f.get_frame()
#             ims.append(fix(im))

72984


In [103]:
SeqProcess.save_gif(ims,'/home/david/test.gif',1/25)